Scraping for playing probability for the current Serie A matchday.

Data from http://fantacalcio.it

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [2]:
res = requests.get('https://www.fantacalcio.it/probabili-formazioni-serie-a')

In [3]:
soup = BeautifulSoup(res.text)

groups = soup.findAll('ul', attrs = {'class' : 'player-list'})

probables = pd.DataFrame(columns = ['player', 'starter', 'percentage'])

for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    bars = groups[i].findAll('div', attrs = {'class' : 'progress-bar'})
    
    starter_string = groups[i]['class'][1]
    if(starter_string == 'starters'):
        starter = 1
    else:
        starter = 0
    
    for j in range(len(players)):
        player = players[j].contents[1].contents[0]
        perc = float(bars[j]['aria-valuenow'])
        
        row_df = pd.DataFrame([[player, starter, perc]], columns = probables.columns)
        
        probables = pd.concat([probables, row_df], ignore_index = True)
    
probables   

,player,starter,percentage
0,Tatarusanu,1,90.0
1,Thiaw,1,65.0
2,Kjaer,1,85.0
3,Kalulu,1,90.0
4,Saelemaekers,1,60.0
...,...,...,...
433,Dumfries,0,60.0
434,Mkhitaryan,0,55.0
435,Asllani,0,30.0
436,Gagliardini,0,50.0


In [4]:
probables = probables.set_index('player')

In [5]:
groups = soup.findAll('ul', attrs = {'class' : 'ballot-list'})

ballots = pd.DataFrame(columns = ['player1', 'player2', 'percentage'])
for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    percs = groups[i].findAll('strong', attrs = {'class' : 'percentage'})
    
    player1 = players[0].contents[1].contents[0]
    player2 = players[1].contents[1].contents[0]
    perc = float(percs[0].contents[0].replace('%', ''))
    
    row_df = pd.DataFrame([[player1, player2, perc]], columns = ballots.columns)
        
    ballots = pd.concat([ballots, row_df], ignore_index = True)
                                  
ballots

,player1,player2,percentage
0,Saelemaekers,Calabria,60.0
1,Diaz B.,De Ketelaere,60.0
2,Thiaw,Messias,65.0
3,Singo,Aina,55.0
4,Rodriguez R.,Buongiorno,55.0
5,Akpa Akpro,Haas,60.0
6,Satriano,Piccoli,60.0
7,Ebuehi,Stojanovic,60.0
8,Holm,Ferrer,55.0
9,Nzola,Shomurodov,55.0


In [6]:
for i in range(ballots.shape[0]):
    p1 = ballots['player1'][i]
    p2 = ballots['player2'][i]
    
    perc = float(ballots['percentage'][i])
    
    probables.at[p1, 'starter'] = perc/100
    probables.at[p2, 'starter'] = 1 - perc/100
    
probables

,starter,percentage
player,,
Tatarusanu,1,90.0
Thiaw,0.65,65.0
Kjaer,1,85.0
Kalulu,1,90.0
Saelemaekers,0.6,60.0
...,...,...
Dumfries,0.45,60.0
Mkhitaryan,0.45,55.0
Asllani,0,30.0


In [7]:
probables.to_excel('mid_outputs/match_probable_players.xlsx')   